In [1]:
TRAIN_DATA_PATH = "../../data/wikipedia/train.csv"
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
comment_col = 'comment_text'

In [2]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import re

Using Theano backend.


In [3]:
train = pd.read_csv(TRAIN_DATA_PATH)
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [32]:
train["comment_text"] = train["comment_text"].apply(lambda x: x.lower())
train["comment_text"] = train["comment_text"].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
for i in label_cols:
    print("Label: "+str(i))
    print("Number of positive examples " + str(train[train[i] == 0].size))    
    print("Number of negative examples " + str(train[train[i] == 1].size))
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train['comment_text'].values)
X = tokenizer.texts_to_sequences(train['comment_text'].values)
X = pad_sequences(X)

Label: toxic
Number of positive examples 1154216
Number of negative examples 122352
Label: severe_toxic
Number of positive examples 1263808
Number of negative examples 12760
Label: obscene
Number of positive examples 1208976
Number of negative examples 67592
Label: threat
Number of positive examples 1272744
Number of negative examples 3824
Label: insult
Number of positive examples 1213552
Number of negative examples 63016
Label: identity_hate
Number of positive examples 1265328
Number of negative examples 11240


In [38]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss = 'mean_squared_error', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/layers/recurrent.py:1993: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1396, 128)         256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 1396, 128)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 197       
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X,train["toxic"], test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(106912, 1396) (106912,)
(52659, 1396) (52659,)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7


Reference: https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras 